In [1]:
import scipy.integrate as integrate

#### Berekenen momentcapaciteit [kNm]:

In [1]:
def calc_M_Rd(f_cd,f_yd,b,h,A_s):
    d = h - a
    rho_1 = A_s / (b * h)
    x_u = (rho_1 / alpha) * (f_yd/f_cd) * d
    z = d - beta * x_u
    M_Rd = z * f_yd * A_s / 1e6
    
    return M_Rd, z

#### Berekenen dwarskrachtcapaciteit [MPa]:

In [ ]:
def v_Rdc(d, rho_1, f_ck, sigma_cp): # d [mm] / rho_1 [%] / f_ck [MPa] / sigma_cp [MPa]
    C_Rdc = 0.18 / g_c
    k1 = 0.15
    rho_1 = rho_1 / 100
    k = min(1 + (200/d)**0.5,2)
    
    v_Rdc1 = C_Rdc * k * (100 * rho_1 * f_ck)**(1/3) + k1 * sigma_cp
    v_Rdc2 = 0.035 * k**(3/2) * f_ck**(1/2) + k1 * sigma_cp
    
    v_Rdc = max(v_Rdc1, v_Rdc2)
    return v_Rdc

#### Bereken rek- en spanningsverloop op basis van rekken in uitertse vezels:

In [1]:
def eps(z, e1, e2):
    return (e2 - e1) / h * z + e1

def func_Nb(z, e1, e2, f_cd):
    return b * sig_c(z, e1, e2, f_cd)

def func_zb(z, e1, e2, f_cd):
    return z * func_Nb(z, e1, e2, f_cd)

def sig_c(z, e1, e2, f_cd):
    if eps(z, e1, e2) > 0:
        s = 0
    elif eps(z, e1, e2) < 0 and eps(z, e1, e2) > eps_c_el:
        s = eps(z, e1, e2) / eps_c_el * -f_cd
    else:
        s = -f_cd

    return s

In [ ]:
def sig_c_bak(z, e1, e2, f_cd):
    sig = []
    for i in range(len(z)):
        if eps(z[i], e1, e2) > 0:
            sig = np.append(sig, 0)
        elif eps(z[i], e1, e2) < 0 and eps(z[i], e1, e2) > eps_c_el:
            sig =np.append(sig, eps(z[i], e1, e2) / eps_c_el * -f_cd)
        else:
            sig = np.append(sig, -f_cd)
    return sig

#### Berekening interne en externe snedekrachten op basis van rekken:

In [61]:
def MN_int(e1, e2):
    N_b = integrate.quad(func_Nb, 0, h, args=(e1,e2,f_cd))[0]
    if N_b == 0:
        z_b = 0.5 * h
    else:
        z_b = integrate.quad(func_zb, 0, h, args=(e1,e2,f_cd))[0] / N_b
    
    N_b = N_b / 10**3
    
    eps_s1 = eps(ys1, e1, e2)
    eps_s2 = eps(ys2, e1, e2)
    N_s1 = eps_s1 * E_sd(eps_s1) * As1 / 10**3 # [kN]
    N_s2 = eps_s2 * E_sd(eps_s2) * As2 / 10**3 # [kN]
    return [N_b, z_b, N_s1, N_s2]

def MN_ext(eps_1, eps_2):
    kappa = (eps_2 - eps_1) / (h / 10**3)
    [N_b, z_b, N_s1, N_s2] = MN_int(eps_1, eps_2)
    N_res = N_b + N_s1 + N_s2
    e_res = (N_b * z_b + N_s1 * ys1 + N_s2 * ys2) / N_res - 0.5 * h
    M_res = e_res / 10**3 * N_res
    res = [N_res, e_res, M_res, kappa]

    return res

#### Capaciteitscurve

In [2]:
def MNCapCurve(b, h, ys1, As1, ys2, As2, n): # Respectievelijk breedte / hoogte / staafafstand zijde 1 / wapeningsopp zijde 1 / staafafstand zijde 2 /  wapeningsopp zijde 2 / stappen per curvedeel
    A_c = b * h
    ys1 = as1 # [mm]
    ys2 = h - as2 # [mm]
    
    # Plastische capaciteit
    eps_1_s_el = eps_c_pl + h / (h - ys1) * (eps_s_el - eps_c_pl)
    eps_1_s_pl = eps_c_pl + h / (h - ys1) * (eps_s_pl - eps_c_pl)
    eps_2_s_el = eps_c_pl + h / ys2 * (eps_s_el - eps_c_pl)
    eps_2_s_pl = eps_c_pl + h / ys2 * (eps_s_pl - eps_c_pl)

    eps_1_stps = [eps_c_pl, 0, eps_1_s_el, eps_1_s_pl, eps_s_el, eps_c_pl, eps_c_pl, eps_c_pl, eps_c_pl]
    eps_2_stps = [eps_c_pl, eps_c_pl, eps_c_pl, eps_c_pl, eps_s_el, eps_2_s_pl, eps_2_s_el, 0, eps_c_pl]

    eps_1 = add_steps(n, eps_1_stps)
    eps_2 = add_steps(n, eps_2_stps)

    N_Rd = []
    e_M = []
    M_Rd = []
    kappa = []

    for i in range(len(eps_1)):
        N_Rd = np.append(N_Rd, MN_ext(eps_1[i], eps_2[i])[0])
        e_M = np.append(e_M, MN_ext(eps_1[i], eps_2[i])[1])
        M_Rd = np.append(M_Rd, MN_ext(eps_1[i], eps_2[i])[2])
        kappa = np.append(kappa, MN_ext(eps_1[i], eps_2[i])[3])
    
    # Elastische capaciteit
    eps_1_s_el = eps_c_el + h / (h - ys1) * (eps_s_el - eps_c_el)
    eps_2_s_el = eps_c_el + h / ys2 * (eps_s_el - eps_c_el)

    eps_1_stps = [eps_c_el, 0, eps_1_s_el, eps_s_el, eps_c_el, eps_c_el, eps_c_el]
    eps_2_stps = [eps_c_el, eps_c_el, eps_c_el, eps_s_el, eps_2_s_el, 0, eps_c_el]

    eps_1 = add_steps(n, eps_1_stps)
    eps_2 = add_steps(n, eps_2_stps)

    N_Rd_el = []
    e_M_el = []
    M_Rd_el = []
    kappa_el = []

    for i in range(len(eps_1)):
        N_Rd_el = np.append(N_Rd_el, MN_ext(eps_1[i], eps_2[i])[0])
        e_M_el = np.append(e_M_el, MN_ext(eps_1[i], eps_2[i])[1])
        M_Rd_el = np.append(M_Rd_el, MN_ext(eps_1[i], eps_2[i])[2])
        kappa_el = np.append(kappa_el, MN_ext(eps_1[i], eps_2[i])[3])
    
    return N_Rd, M_Rd, N_Rd_el, M_Rd_el

#### Bepalen snijpunt van 2 functies (algemeen):

In [4]:
def find_roots(x,y):
    s = np.abs(np.diff(np.sign(y))).astype(bool)
    return x[:-1][s] + np.diff(x)[s]/(np.abs(y[1:][s]/y[:-1][s])+1)

#### Zoeken naar oplossing voor meerdere vergelijkingen:

In [64]:
def equations(p):
    x, y = p
    return (MN_ext(x,y)[0] - N_Ed, MN_ext(x,y)[2] - M_Ed)

#### Vergroot resolutie array:

In [67]:
def add_steps(n, arr):
    i = 0
    arr_new = []
    while i < len(arr) - 1:
        d_a = arr[i+1] - arr[i]
        a = np.linspace(arr[i], arr[i+1], n + 1)
        if i > 0:
            a = a[1:]
        arr_new = np.append(arr_new,a)            
        #print(a)
        i += 1

    return arr_new